# This notebook demonstrates how to access data on Microsoft Planetary Computer using STAC API

This notebook includes steps to access data on MPC using the arcpy STAC API. Key steps include:
1. Finding the data collection of interest and the corresponding cloud storage connection file (if required)
2. Defining a query to search for data in the collection
3. Accessing search results

#### Estimated running time: 2 minutes

In [ ]:
import arcpy
from arcpy import AIO

### A cloud storage connection file is often required to access the data

Check this link for the list of collections and whether ACS file is required:
<br>
https://pro.arcgis.com/en/pro-app/latest/help/data/imagery/supported-stac-collections.htm

In [ ]:
# Create an AIO object from the cloud storage connection file for data access
a = AIO(r'C:\AMPC_Resources\ACS_Files\esrims_pc_landsat-c2-l2.acs')

### A query is required to search for data of interest.

A commonly used query consists of 5 keywords:
1. collections: name of the collection requested
2. bbox: the geographical bounding box of the search in the form of 
<br>[Lower left corner X, Lower left  corner Y, Upper right corner X, Upper right corner Y] in WGS84 coordinates
3. query: text search for collection metadata
<br>metadata can be found under each collection's API site.
<br>e.g. Landsat collection 2 level 2: https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2
4. datetime: time range of the search
5. limit: the maximum number of results to be returned

More details on query parameters can be found here:
<br>
https://docs.ogc.org/DRAFTS/20-004.html#core-query-parameters
<br>
https://github.com/stac-api-extensions/query

In [ ]:
# Example query for Landsat collection 2 level 2 product
query = {
    "collections": ["landsat-c2-l2"], # collection id of Landsat collection 2 level 2 product (https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)
    "bbox": [-156.1271927, 18.8710955, -154.7755921, 20.2990569], # bounding box over Hawaii island
    "query": {"platform": {"in": ["landsat-8"]}}, # query to only return Landsat-8 images
    "datetime": "2022-01-01/2022-12-31", # time range
    "limit": 10, # max number of results to be returned
}

### attribute_dict is an optional parameter that specifies what metadata you want to return to the RasterCollection.

metadata can be found under each collection's API site
<br>
e.g. Landsat collection 2 level 2: https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2 

In [ ]:
# Example attribute_dict for Landsat collection 2 level 2 product
attribute_dict = {
    "Name":"id",
    "Cloud Cover":"eo:cloud_cover",
    "StdTime":"datetime",
    "Platform":"platform",
    "Spatial Reference":"proj:epsg",
    "Extent": "bbox",
}

### A RasterCollection is required to return the search result in a usable container

In [ ]:
# Create a RasterCollection object that contains the search results
rc = arcpy.ia.RasterCollection.fromSTACAPI(stac_api="https://planetarycomputer.microsoft.com/api/stac/v1",
                                           query=query,
                                           attribute_dict=attribute_dict)

In [ ]:
rc

### You can also customize the data returned to the RasterCollection using the context parameter

Available keys are processingTemplate and assetManagement
<br>
1. processingTemplate.
    This specifies the template used to pre-process the raw data or combinations of the raw data that is returned to the RasterCollection. The default is Multiband.
<br>Available processing templates can be found: https://pro.arcgis.com/en/pro-app/3.3/help/data/imagery/satellite-sensor-raster-types.htm

2. assetManagement. This specifies the individual assets to be returned to the RasterCollection. If multiple assets are selected, the collection will be composed of multiband rasters from those selected asset types.

In [ ]:
# Create a RasterCollection object with processingTemplate applied
rc_template = arcpy.ia.RasterCollection.fromSTACAPI(stac_api="https://planetarycomputer.microsoft.com/api/stac/v1",
                                           query=query,
                                           attribute_dict=attribute_dict,
                                           context = {"processingTemplate": "Surface Reflectance"})

In [ ]:
rc_template[0]['Raster'].bandNames

In [ ]:
# Create a RasterCollection object with assetManagement applied
rc_asset = arcpy.ia.RasterCollection.fromSTACAPI(stac_api="https://planetarycomputer.microsoft.com/api/stac/v1",
                                           query=query,
                                           attribute_dict=attribute_dict,
                                           context = {"assetManagement": ["red", "green", "blue"]})

In [ ]:
# Comparing the different band combinations returned to the 3 raster collections
print(f"{'Default template (Multiband):' :<30} {rc[0]['Raster'].bandNames}")
print(f"{'Surface Reflectance template:' :<30} {rc_template[0]['Raster'].bandNames}")
print(f"{'Specific assets:' :<30} {rc_asset[0]['Raster'].bandNames}")